# Generics

In this topic we are going to explore new concepts such reified type parameters and declaration-site variance.

_Refied type parameters_ allow us to refer at runtime to the specific types used as type arguments in an inline function call.

_Declaration-site variance_ lets us specify whether a generic type with a type argument is a subtype or a supertype of another generic type with the same base type and a different type of argument. For example, it regulates whether it's possible to pass arguement of type List<Int> to function expecting List<Any>.

## Generic Functions and Properties

Lets say we want to write a function that work with a list, we want it to work with any list, not just a list of elements of a specific type and for that we need to write a generic function.


### Generic Function Declaration

<img src="../resources/generic-example-1.png"/>


The function's type parameter T is used in the receiver type and in the return type; both of them are List<T>. 

In [ ]:
val authors = listOf("Dmitry", "Svetlana")
val readers = mutableListOf<String>()

fun <T> List<T>.filter(predicate: (T) -> Boolean ): List<T>

readers.filter { it !in authors }

In the above example the type of the autogenerated lambda parameter it is String in this case. The compiler has to infer that: after all, in the declaration of the function, the lambda parameters has a generic type T (it's the type of the function arguement in (T) -> Boolean). The compiler understand that T is String, because it knows the function should be called on List<T>, and the actual type of its receiver, readers, is List<String>

### Declaring generic classes

We declare a generic class in kotlin by putting angle brackets after the class name and the type parameters in the angle brackets. Once we do thatm we can use the type parameters in the body of the class, just like any other types.

``` kotlin
interface List<T> {
    operator fun get(index: Int) : T
}
```


If our class extends a generic class (or implements a generic interface), we have to provide a type argument for the generic parameter of the base type. It can be either a specific type or another type parameter:

``` kotlin
class StringList: List<String> {
    override fun get(index: Int): String =  ...
}

class ArrayList<T>: List<T> {
    override fun get(index: Int): T = 
}
```


### Type parameter constraint

_Type Parameter Constraints_ as the name suggest let us restrict the types that can be used as type arguments for a class or function. For example, consider a function that calculates the sum of elements in a list.it can be used on a `List<Int>` or a `List<Double>`, but not, for example, a `List<String>`. To express this, you can define a type parameter constraint that specifies that the type parameter of `sum` must be a number

<img src="../resources/generic-example-2.png"/>

The actual type argument should extend `Number` to allow this function invocation

Once we specified a bound for a type parameter T, we can use values to type T as values of its upper bound.

In [2]:
fun <T: Number> oneHalf(value: T): Double {
    return value.toDouble() / 2.0
}

println(oneHalf(3))

1.5


Lets write a generic function that finds the maximum of two items. Because its possible to find a maximum of items that can be compared to each other, we need to specify that in the signature of the function.

In [3]:
fun <T: Comparable<T>> max(first: T, second: T): T {
    return if (first > second) first else second
}

println(max("kotlin", "java"))

kotlin


In the rare case when we need to specify multiple constraints on a type parameter, we can a slightly different syntax

In [4]:
fun <T> ensureTrailingPeriod(seq: T)
    where T: CharSequence, T: Appendable {
        if(!seq.endsWith('.')){
            seq.append('.')
        }
}

### Making type parametes non-null

If you declare a generic class or function, any types, including nullable ones, can be substituted for its type parameters.

In [5]:
class Processor<T>{
    fun process(value: T){
        value?.hashCode()
    }
}

In the process function, the parameter value is nullable, even though T isn't marked with a question mark. This is the case because specific instantiations of the Processor class can use a nullable type of T:

In [6]:
val nullableStringProcessor = Processor<String?>()
nullableStringProcessor.process(null)

If we want to guarantee that a non-null type will always be substituted for a type parameter, we can achieve this by specifying a constraint.

``` kotlin
class Processor<T: Any> {
    fun process(value: T){
        value.hashCode()
    }
}
```

## GENERICS AT RUNTIME: ERASED AND REIFEID TYPE PARAMETRS

### Generics at runtime: type checks and casts

Kotlin's generics are erased at runtime. this means an instance of generic class doesn't carry information about the type arguements used to create that instance For example, if you create a List<String> and put a bunch of strings into it, at runtime you’ll only be able to see that it’s a List. It’s not possible to identify which type of elements the list was intended to contain. (Of course, you can get an element and check its type, but that won’t give you any guarantees, because other elements may have slightly different types.

In [6]:
val value: List<String> = listOf("a", "b")
val list2: List<Int> = listOf(1, 2, 3)

if (value is List<Int>){
    println("Value is List of String")
}

Cannot check for instance of erased type: List<Int>

Even though  it's perfectly possible to find out at runtime that value is a List, but we can't tell whether its a list of Person or a String.

To check whether a given variable is a List not Set or something else we can use

In [7]:
if (value is List<*>){
    println("Value is a list")
}

Value is a list


We can still use normal generic types in as and as? cast. But the cast won't fail if the class has the correct base type and wrong type arguement because the type arguement isn't known at runtime when the cast is performed.

In [10]:
fun printSum(c: Collection<*>){
    val intList = c as? List<Int>
        ?: throw IllegalArgumentException("List is expected")
        
    println(intList.sum())
}

printSum(listOf(1,2,3))
printSum(listOf("a","b","c"))

6


java.lang.ClassCastException: java.lang.String cannot be cast to java.lang.Number
kotlin.collections.CollectionsKt___CollectionsKt.sumOfInt(_Collections.kt:2474)
Line_10_jupyter.printSum(Line_10.jupyter.kts:5)
Line_10_jupyter.<init>(Line_10.jupyter.kts:9)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(Unknown Source)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(Unknown Source)
java.lang.reflect.Constructor.newInstance(Unknown Source)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:95)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:40)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvmhost.repl.JvmReplEvaluator$eval$$inlined$write$lambda$2.invokeSuspend(legacyReplEvaluation.kt:57)
kotlin.coroutines.jvm.inte

You don’t get an IllegalArgumentException, because you can’t check whether the argument is a List<Int>. Therefore the cast succeeds, and the function sum is called on such a list anyway. Although it’s executed, an exception is thrown. This happens because the function tries to get Number values from the list and add them together. An attempt to use a String as a Number results in a ClassCastException at runtime.